# Was sind RDD

RDDs sind die grundlegende Datenstruktur von Spark. Es sind Resilient Distributed Datassets.

[Link zur Dokumentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html)

![](RDD-basics.dio.svg)

Wie mappt das auf das Ausführungsmodell von Spark?

![](RDD-mapping-on-spark.dio.svg)


## Haupteigenschaften von RDDs

* **In-Memory**
* **Partitioniert** (alle Partitionen zusammen sind ein RDD, sind sind üblicherweise im Speicher des Clusters verteilt.
* **Read-Only** jede Änderung erzeugt einen neuen RDD
* **Resillient** können im Fehlefall sich recovern

## Was ist der Lineage Graph

![](RDD-lineage-graph.dio.svg)

Er enthält die verschiedenen Schritte die auf die einzelnen Partitionen angewendet werden.

# Erste Schritte mit RDDs

## Initialisierung von Spark

In [1]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession

In [4]:
spark = (
        SparkSession
         .builder
         .appName("RDD-First-Steps")
         .master("local[4]")
         .getOrCreate()
        )
sc = spark.sparkContext # nützlich um z.B. RDDs innerhalb einer Spark-Session zu erzeugen
spark

## Erzeuge deinen ersten RDD

* [PySpark Docu](https://spark.apache.org/docs/latest/api/python/getting_started/index.html)
* [RDD Python API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.html#rdd-apis)

In [9]:
number_rdd = sc.parallelize([1, 2, 3, 4, 5])

In [10]:
type(number_rdd)

pyspark.rdd.RDD

In [11]:
# wieviele Partitionen wurden erzeugt
number_rdd.getNumPartitions()

4

In [12]:
# alles ausgeben
number_rdd.collect()

[1, 2, 3, 4, 5]

In [13]:
# nur den ersten Eintrag ausgeben
number_rdd.first()

1

In [14]:
# die ersten zwei Einträge ausgeben
number_rdd.take(2)

[1, 2]

In [18]:
# auf gerade Zahlen filtern
even_number_rdd = number_rdd.filter(lambda x : x % 2 == 0)
even_number_rdd.collect()

[2, 4]

In [30]:
# ein leicht komplexeres Beispiel
address_rdd = sc.parallelize(
    [
        [1, "Marc", "Dortmund"],
        [2, "Bernd", "Hamburg"]
    ]
)

In [31]:
address_rdd.first()

[1, 'Marc', 'Dortmund']

In [32]:
people_from_hamburg_rdd = address_rdd.filter(lambda x: x[2]=='Hamburg')

In [33]:
people_from_hamburg_rdd.collect()

[[2, 'Bernd', 'Hamburg']]

## Erzeugen von RDDs aus Dateien


In [35]:
taxi_zones_rdd = sc.textFile("TaxiZones.csv") # Du könntest auch Ordner angeben. Dann würden die ganzen Ordner gelesen.

In [36]:
taxi_zones_rdd

TaxiZones.csv MapPartitionsRDD[26] at textFile at NativeMethodAccessorImpl.java:0

In [37]:
# die ersten 10 Einträge ausgeben
taxi_zones_rdd.take(10)

['1,EWR,Newark Airport,EWR',
 '2,Queens,Jamaica Bay,Boro Zone',
 '3,Bronx,Allerton/Pelham Gardens,Boro Zone',
 '4,Manhattan,Alphabet City,Yellow Zone',
 '5,Staten Island,Arden Heights,Boro Zone',
 '6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone',
 '7,Queens,Astoria,Boro Zone',
 '8,Queens,Astoria Park,Boro Zone',
 '9,Queens,Auburndale,Boro Zone',
 '10,Queens,Baisley Park,Boro Zone']

In [38]:
taxi_zones_rdd.getNumPartitions()

2

Wieso könnte Spark das gemacht haben?

In [39]:
# Nun lesen und 4 Partitionen erzwingen
taxi_zones_rdd = sc.textFile("TaxiZones.csv", 4)
taxi_zones_rdd.getNumPartitions()

4

## Eine der wichtigsten Operationen: Map

In [43]:
taxi_zones_splitted = (
    taxi_zones_rdd
        .map(lambda row: row.split(","))
)
taxi_zones_splitted.take(5)

[['1', 'EWR', 'Newark Airport', 'EWR'],
 ['2', 'Queens', 'Jamaica Bay', 'Boro Zone'],
 ['3', 'Bronx', 'Allerton/Pelham Gardens', 'Boro Zone'],
 ['4', 'Manhattan', 'Alphabet City', 'Yellow Zone'],
 ['5', 'Staten Island', 'Arden Heights', 'Boro Zone']]

In [46]:
filtered_zones_rdd = (
    taxi_zones_splitted
        .filter(lambda row: row[1] == "Manhattan" and row[2].lower().startswith("central"))
)
filtered_zones_rdd.take(5)


[['41', 'Manhattan', 'Central Harlem', 'Boro Zone'],
 ['42', 'Manhattan', 'Central Harlem North', 'Boro Zone'],
 ['43', 'Manhattan', 'Central Park', 'Yellow Zone']]

**Aufgabe** Filter nach 2. Element ist Bronx

und/oder

Filter nach geraden IDs